In [1]:
import sys
sys.path.append("../../")

In [2]:
import numpy as np
from tqdm import tqdm

# Load Dataset

In [3]:
from graphgallery.data import Planetoid, NPZDataset;
data = Planetoid('cora', root="~/GraphData/datasets/", verbose=False)
graph = data.graph
idx_train, idx_val, idx_test = data.split_nodes()

# Init

In [4]:
table = []
device = "gpu"
runs = 10

GNNs = ("GCN", "SGC", "GAT")
Epochs = (200, 100, 200)
Backends = ("TensorFlow", "PyTorch", "PyG", "DGL")

# Benchmarking Experiments

In [6]:
from graphgallery.nn import gallery
from graphgallery import set_backend

for backend in Backends:
    set_backend(backend)
    tmp = []
    for Net, epochs in zip(GNNs, Epochs):
        results = []
        for seed in range(runs):
            model = getattr(gallery, Net)(graph, attr_transform="normalize_attr", 
                                          device=device, seed=42+seed)
            model.build()
            his = model.train(idx_train, idx_val, verbose=0, epochs=epochs)
            loss, accuracy = model.test(idx_test, verbose=0)
            results.append(accuracy)

        results = f"{np.mean(results):.2%}±{np.std(results):.2%}"
        print(f"{model}: \033[31m{results}\033[0m")        
        tmp.append(results)
    table.append(tmp)


GCN(device=GPU:0, backend=TensorFlow 2.1.2 Backend): 80.48%±0.97%
SGC(device=GPU:0, backend=TensorFlow 2.1.2 Backend): 81.33%±0.61%
GAT(device=GPU:0, backend=TensorFlow 2.1.2 Backend): 81.96%±1.02%
GCN(device=cuda:0, backend=PyTorch 1.6.0+cu101 Backend): 81.42%±0.70%
SGC(device=cuda:0, backend=PyTorch 1.6.0+cu101 Backend): 80.53%±0.48%
GAT(device=cuda:0, backend=PyTorch 1.6.0+cu101 Backend): 82.74%±0.58%
GCN(device=cuda:0, backend=PyTorch Geometric 1.6.1 (PyTorch 1.6.0+cu101) Backend): 81.22%±0.75%
SGC(device=cuda:0, backend=PyTorch Geometric 1.6.1 (PyTorch 1.6.0+cu101) Backend): 79.90%±1.17%
GAT(device=cuda:0, backend=PyTorch Geometric 1.6.1 (PyTorch 1.6.0+cu101) Backend): 82.72%±1.08%


Using backend: pytorch


GCN(device=cuda:0, backend=DGL PyTorch 0.5.2 (PyTorch 1.6.0+cu101) Backend): 81.39%±0.44%
SGC(device=cuda:0, backend=DGL PyTorch 0.5.2 (PyTorch 1.6.0+cu101) Backend): 80.70%±0.54%
GAT(device=cuda:0, backend=DGL PyTorch 0.5.2 (PyTorch 1.6.0+cu101) Backend): 82.18%±0.47%


# Results

In [7]:
import pandas as pd
table = pd.DataFrame(table, columns=GNNs)
table.index = Backends
table

GCN           SGC           GAT
TensorFlow  80.48%±0.97%  81.33%±0.61%  81.96%±1.02%
PyTorch     81.42%±0.70%  80.53%±0.48%  82.74%±0.58%
PyG         81.22%±0.75%  79.90%±1.17%  82.72%±1.08%
DGL         81.39%±0.44%  80.70%±0.54%  82.18%±0.47%